In [3]:
%autosave 60

Autosaving every 60 seconds


In [4]:
%load_ext autoreload
%autoreload 2

In [70]:
import sys
import json
import ollama
import torch
import datasets
import transformers
from IPython.display import clear_output
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM

In [53]:
# понадобится для оценки модели
# pip install human-eval==1.0.3
# pip install ollama==0.5.1

In [12]:
print(sys.version)

3.13.5 (main, Jun 11 2025, 15:36:57) [Clang 17.0.0 (clang-1700.0.13.3)]


In [55]:
print('torch', torch.__version__)
print('datasets', datasets.__version__)
print('transformers', transformers.__version__)

torch 2.7.1
datasets 3.6.0
transformers 4.46.0


# Семинар 4, Часть 2: AI Coding Assistants в Enterprise
## От автодополнения до автономных агентов-разработчиков

Сегодня мы погрузимся в мир AI-ассистентов для разработки. Эти инструменты эволюционировали от простых подсказчиков до мощных систем, способных понимать весь проект и выполнять сложные задачи автономно. 

На этом семинаре мы:
1.  Рассмотрим ключевых игроков на рынке AI-ассистентов.
2.  Разберем 5 уровней "интеллекта" этих инструментов.
3.  Заглянем "под капот" и разберем технологии, которые делают их возможными: от Fill-in-the-Middle до RAG и агентных фреймворков.
4.  Напишем код, иллюстрирующий эти ключевые концепции.


## Обзор существующих решений

Рынок AI-ассистентов для разработки быстро растет. Рассмотрим несколько популярных и интересных примеров, которые представляют разные подходы к помощи разработчику.

| Инструмент | Ключевая особенность | Модель использования | Ссылка |
|---|---|---|---|
| **GitHub Copilot** | Глубокая интеграция с экосистемой GitHub и VS Code. Стандарт индустрии. | Плагин в IDE | [Website](https://github.com/features/copilot) / [GIT](https://github.com/microsoft/vscode-copilot-chat)|
| **Cursor** | Форк VS Code, "AI-first" IDE с нативным чатом и анализом всего репозитория. | Stand-alone IDE | [Website](https://cursor.sh/) |
| **Continue** | Open-source и кастомизируемая платформа. Позволяет подключать свои модели (в т.ч. локальные). | Плагин в IDE | [GitHub](https://github.com/continuedev/continue) / [GIT](https://github.com/continuedev/continue)|
| **Void** | Privacy-focused альтернатива Cursor с поддержкой локальных LLM. | Stand-alone IDE | [GitHub](https://github.com/voideditor/void) / [GIT](https://github.com/voideditor/void)|
| **Cline** | Автономный агент для VS Code с доступом к терминалу для выполнения задач. | Плагин в IDE | [GitHub](https://github.com/cline/cline) / [GIT](https://github.com/cline/cline)|


## Пять уровней "помощи" AI-ассистента

Чтобы лучше понять эволюцию и возможности этих инструментов, можно выделить 5 условных уровней их "интеллекта" и автономности:

1. **Прямая генерация кода (Autocomplete)**
   *Самая известная функция: вы начинаете писать код, и ассистент предлагает завершение строки или целый блок кода. Это быстро и интуитивно.*

2. **Интерактивный чат и генерация по запросу**
   *Вы можете "поговорить" с ассистентом: выделить код и спросить: "Объясни, что здесь происходит", "Найди ошибку" или попросить сгенерировать функцию по описанию.*

3. **Автоматический рефакторинг и "умные" правки**
   *Ассистент может предложить улучшенную версию кода (например, более читаемую или производительную). Изменения отображаются в виде diff-сравнения.*

4. **Понимание всего проекта (Full Codebase Awareness)**
   *Ассистент дает ответы, учитывая весь ваш проект, а не только открытые файлы. Он знает о ваших кастомных классах и архитектуре.*

5. **"Агенты" и автономные задачи (Code Agents)**
   *Вы ставите высокоуровневую задачу: "Добавь аутентификацию через GitHub". Агент сам составляет план, находит и редактирует нужные файлы, пишет код в нескольких местах и даже может запустить тесты для проверки.*


## Как это работает под капотом

### Уровень 1: Инлайновые подсказки и Fill-in-the-Middle
**Как это работает?** 

- **Триггер**: Срабатывает автоматически после паузы во время набора текста.
- **Сбор контекста**: Система собирает очень ограниченный, но релевантный контекст: текст до и после курсора в текущем файле. Скорость здесь — главный приоритет.
- **Формирование промпта (Fill-in-the-Middle)**: Вместо простого дополнения текста, современные модели используют технику FIM. Они получают на вход начало файла (префикс) и конец файла (суффикс), а их задача — сгенерировать недостающую середину. Это позволяет им лучше учитывать контекст с обеих сторон от курсора.
- **Запрос к модели**: Запрос отправляется к легковесной, быстрой LLM, оптимизированной для генерации кода (например, StarCoder, Code Llama, DeepSeek Coder).
- **Отображение**: Полученный ответ мгновенно отображается в редакторе как неактивный текст.



### Задача Fill-in-the-Middle (FIM)

- **Цель:** Дать модели фрагменты кода — префикс (prefix) и суффикс (suffix) — а модель должна сгенерировать «средний» пропущенный код (middle), так чтобы весь код $$prefix + middle + suffix$$ был синтаксически корректным и семантически релевантным.
- Такой подход позволяет лучше учитывать контекст с обеих сторон от места вставки по сравнению с классическим left-to-right (L2R) автодополнением.

### Обучение модели: подготовка данных и трансформация

- Исходный документ кода случайным образом делится на 3 части:
  - Префикс (prefix) — начало документа или кода перед вставкой
  - Средняя часть (middle) — код, который модель должна сгенерировать
  - Суффикс (suffix) — код после вставки
- Создается трансформированный обучающий пример вида:

$$
\text{input} = [\text{prefix}], [\text{suffix}] \quad\rightarrow\quad \text{output} = [\text{middle}]
$$

- Специальные токены и разделители обычно выделяют границы частей (например, `[PRE]`, `[SUF]`, `[MID]`).


<div style="text-align: center;">
  <img src="docs/fitm_1.png" alt="alt text" style="width:400px;">
</div>


### Метрики качества генерации

- **Negative Log-Likelihood (NLL)** – основная метрика, оценивающая вероятность сгенерированной модели последовательности среднего фрагмента.
- **Доля синтаксически корректных сгенерированных фрагментов (Syntax Validity Rate)**
- **Точное совпадение (Exact Match, EM)** – процент сгенерированных средних частей, полностью совпадающих с целевыми.
- **Качество интеграции (fluency, coherence)** – субъективная или автоматическая оценка качества стыковки с префиксом и суффиксом.


Важно проводить оценку не только по общим метрикам, но и по тому, насколько хорошо модели восстанавливают синтаксические структуры на уровне AST (Abstract Syntax Tree).

### Оценка по Abstract Syntax Tree:

- **В эксперименте** участки кода маскируются, модель должна "вставить" пропущенное.  
- Коды (реальный и сгенерированный) разбираются в AST, и сравнивают, какие синтаксические элементы (узлы дерева — Class Definition, If Statement, Assignment и др.) модель верно восстанавливает.
- Для разных подходов обучения строятся сравнения — насколько точнее работают FIM-модели в восстановлении конкретных конструкций.

### Интерпретация результата радар-графика:

- Ось соответствует определённому типу AST-узла (см. подписи).
- Радиус — прирост точности воспроизведения по сравнению с базой.
- Линии разных моделей отражают, где и какие типы кода восстанавливаются лучше (например, Call Expression, Dotted Name, Parameters).




<div style="text-align: center;">
  <img src="docs/fitm_2.png" alt="alt text" style="width:400px;">
</div>

Статьи на тему
- [Structure-Aware Fill-in-the-Middle Pretraining for
Code](https://arxiv.org/pdf/2506.00204)
- [Improving FIM Code Completions via Context & Curriculum
Based Learning](https://arxiv.org/pdf/2412.16589v1)

Эталонные датасеты для оценки
- Single-Line Infilling Benchmark — задачи на заполнение одной строки кода между контекстом.
- CrossCodeEval — более сложный датасет с многопоточечными вызовами, зависимостями и распределением кода по файлам.

In [14]:
# попробуем обученную модель fitm на практике

In [15]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Найден и используется графический процессор Apple (MPS).")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Найдена и используется видеокарта NVIDIA (CUDA).")
else:
    device = torch.device("cpu")
    print("GPU не найден, используется CPU.")

Найден и используется графический процессор Apple (MPS).


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoderbase-1b")
model = AutoModelForCausalLM.from_pretrained("bigcode/starcoderbase-1b")
model = model.to(device)

In [17]:
# определение префикса и суффикса
prefix = "def get_weather(city):\n    # get weather from api"
suffix = "\n    return temperature"

In [18]:
# формирование FIM-промпта
# используются специальные токены для FIM
prompt = f"<fim_prefix>{prefix}<fim_suffix>{suffix}<fim_middle>"

In [19]:
print("--- Сформированный промпт для FIM модели ---")
print(prompt.replace('\n', '\\n'))
print("-" * 40)

--- Сформированный промпт для FIM модели ---
<fim_prefix>def get_weather(city):\n    # get weather from api<fim_suffix>\n    return temperature<fim_middle>
----------------------------------------


In [20]:
# токенизация и отправка на устройство
inputs = tokenizer(prompt, return_tensors="pt").to(device)

In [31]:
# генерация с параметрами сэмплирования
outputs = model.generate(
    **inputs,
    max_new_tokens=64,
    do_sample=True,
    temperature=0.2,
    top_p=0.95,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.convert_tokens_to_ids(["<file_separator>"])[0]
)

In [32]:
# декодирование и очистка результата
# убираем входной промпт из сгенерированного вывода
new_tokens = outputs[0][len(inputs["input_ids"][0]):]
generated_code = tokenizer.decode(new_tokens, skip_special_tokens=False)

In [33]:
# очищаем специальные токены, которые могут появиться в выводе
middle_part = generated_code.replace("<|endoftext|>", "").replace("<file_separator>", "").strip()

In [34]:
print("\n--- Сгенерированный код (вставка) ---")
print(middle_part)
print("-" * 40)


--- Сгенерированный код (вставка) ---
weather = requests.get(f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}')
    return weather.json()

def get_temperature(city):
    # get temperature from api
    weather = requests.get
----------------------------------------


In [35]:
print("\n--- Итоговый полный код ---")
final_code = f"{prefix}\n    {middle_part}{suffix}"
print(final_code)
print("-" * 40)


--- Итоговый полный код ---
def get_weather(city):
    # get weather from api
    weather = requests.get(f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}')
    return weather.json()

def get_temperature(city):
    # get temperature from api
    weather = requests.get
    return temperature
----------------------------------------


### Уровень 2: Интерактивный чат и специализированные модели
**Как это работает?**

- **Триггер**: Явное действие пользователя — отправка сообщения в чате.
- **Сбор контекста**: Система собирает выделенный пользователем фрагмент кода, его вопрос на естественном языке и, возможно, содержимое всего активного файла.
- **Формирование промпта**: Создается инструктивный промпт: `"User has selected the following code: ... He asked the following question: ... Analyze and answer."`
- **Запрос к модели**: Запрос отправляется к более мощной, "разговорной" LLM (например, GPT-4, Claude 3, Llama 3). Для задач, связанных с кодом, предпочтительны модели, дообученные на коде, так как они лучше понимают синтаксис и паттерны.
- **Отображение**: Ответ выводится в окне чата.

#### Почему специализированные на коде модели лучше?
Модели, специально обученные на огромных массивах исходного кода (например, `CodeLlama`, `DeepSeek-Coder`), обладают глубоким "пониманием" синтаксических структур, идиом программирования и распространенных алгоритмов. Они с меньшей вероятностью допустят синтаксические ошибки и предложат более релевантные и эффективные решения, чем модели общего назначения такой же размерности.


### Методология обучения и специализации LLM для генерации кода
#### 1.1. Предобучение (Pre-training): Создание Фундаментальной Модели
На этом этапе закладывается основа модели — ее способность к обобщенному представлению синтаксических и семантических конструкций кода.

*   **Цель:** Обучить модель улавливать статистические закономерности и низкоуровневые паттерны в коде посредством самообучения (unsupervised learning).
*   **Данные:** Крупномасштабные текстовые корпусы, состоящие из триллионов токенов исходного кода из публичных репозиториев (например, GitHub), технической документации и специализированных баз данных.
*   **Процесс:**
    *   **Causal Language Modeling (CLM):** Основная задача — предсказание следующего токена в последовательности. Это заставляет модель изучать грамматические правила, синтаксические структуры и идиоматические конструкции языка программирования.
    *   **Code-Aware Tokenization:** Применяются специализированные токенизаторы, которые сохраняют целостность лексем кода (имена переменных, функций, операторы), что критически важно для семантической корректности.
*   **Результат:** Фундаментальная модель (Foundation Model) с обобщенным представлением о синтаксисе и структуре множества языков программирования, но без специализации для выполнения конкретных инструкций.

<div style="text-align: center;">
  <img src="docs/code_llm.png" alt="alt text" style="width:400px;">
</div>

#### 1.2. Специализированное дообучение (Fine-Tuning): адаптация для конкретных задач
На этом этапе фундаментальная модель адаптируется для выполнения узкоспециализированных задач, востребованных в разработке ПО.

*   **Цель:** Повысить производительность модели на конкретных "downstream" задачах, таких как генерация кода по текстовому описанию, автодополнение, отладка и рефакторинг.
*   **Данные:** Курируемые, высококачественные наборы данных, часто в формате "инструкция-ответ". Используются как синтетические данные, так и задачи из отраслевых бенчмарков (HumanEval, MBPP и др.).
*   **Ключевые техники:**
    *   **Supervised Fine-Tuning (SFT):** Модель обучается на размеченных парах "запрос-эталонный код". Это позволяет ей "выровнять" свои способности с ожиданиями пользователя и научиться генерировать код, соответствующий поставленной задаче.
    *   **Instruction Tuning:** Процесс дообучения, направленный на улучшение способности модели следовать сложным инструкциям на естественном языке, включая многошаговые рассуждения и учет ограничений.
*   **Результат:** Специализированная модель, демонстрирующая высокую точность в целевых задачах, генерирующая более идиоматичный и контекстуально релевантный код.

### 2. Как оценивать такие модели

#### 2.1. Бенчмарки

| Бенчмарк | Назначение и фокус | Структура задач | Основная метрика |
| :--- | :--- | :--- | :--- |
| **HumanEval** | Оценка способности к решению алгоритмических задач и генерации функционально корректного кода по текстовому описанию. | 164 задачи на Python, требующие написания тела функции на основе докстринга и сигнатуры. | `pass@k` |
| **MBPP** | Измерение производительности на базовых задачах программирования. | задачи на Python, менее сложных, чем в HumanEval, с фокусом на стандартные операции. | `pass@k` |
| **CodeXGLUE** | Комплексная оценка широкого спектра задач, связанных с кодом: от генерации и перевода до поиска уязвимостей. | Набор из 14 датасетов, покрывающий 10 различных задач в области Code Intelligence. | `CodeBLEU`, `Accuracy` |

#### 2.2. Интерпретация ключевых метрик

*   **`pass@k`**:
    *   **Определение:** Вероятность того, что хотя бы один из `k` сгенерированных моделью кандидатов успешно проходит все предоставленные юнит-тесты.
    *   **Значение:** Прямая и объективная оценка функциональной корректности генерируемого кода. `pass@1` является наиболее строгим вариантом, оценивающим первую же попытку модели.

*   **`CodeBLEU`**:
    *   **Определение:** Адаптированная для кода версия метрики BLEU. Сравнивает сгенерированный код с эталонным, анализируя совпадения n-граммов, синтаксическую структуру (через Abstract Syntax Tree, AST) и поток данных (data-flow).
    *   **Значение:** Оценивает не только лексическое совпадение, но и структурное и семантическое качество кода, приближаясь к оценке, которую дал бы человек-эксперт.

In [41]:
# оценим работу модели qwen2.5-coder
# подробнее о модели вот тут https://ollama.com/library/qwen2.5-coder

In [185]:
# карточка датасета для оценки (вариант 1)
# https://huggingface.co/datasets/openai/openai_humaneval

In [ ]:
# мы будем проверять на датасете попроще: MBPP (вариант 2)
# запустим оценку заранее написанным скриптом

In [191]:
%run src/run_benchmark_mbpp.py --limit -1

✅ Задача 11: успешно
✅ Задача 12: успешно
✅ Задача 14: успешно
✅ Задача 16: успешно
✅ Задача 17: успешно
❌ Задача 18: ошибка
✅ Задача 19: успешно
❌ Задача 20: ошибка
❌ Задача 56: ошибка
✅ Задача 57: успешно
✅ Задача 58: успешно
✅ Задача 59: успешно
✅ Задача 61: успешно
✅ Задача 62: успешно
❌ Задача 63: ошибка
✅ Задача 64: успешно
✅ Задача 65: успешно
✅ Задача 66: успешно
✅ Задача 67: успешно
✅ Задача 68: успешно
✅ Задача 69: успешно
❌ Задача 70: ошибка
✅ Задача 71: успешно
❌ Задача 72: ошибка
✅ Задача 74: успешно
✅ Задача 75: успешно
✅ Задача 77: успешно
✅ Задача 79: успешно
✅ Задача 80: успешно
❌ Задача 82: ошибка
❌ Задача 83: ошибка
✅ Задача 84: успешно
❌ Задача 85: ошибка
✅ Задача 86: успешно
❌ Задача 87: ошибка
✅ Задача 88: успешно
✅ Задача 89: успешно
✅ Задача 90: успешно
✅ Задача 91: успешно
✅ Задача 92: успешно
✅ Задача 93: успешно
✅ Задача 94: успешно
✅ Задача 95: успешно
✅ Задача 96: успешно
✅ Задача 97: успешно
❌ Задача 98: ошибка
✅ Задача 99: успешно
✅ Задача 100: успешно
✅ 

**Это очень хороший результат!** 👍

Для модели такого размера, запущенной на обычном компьютере без специальных настроек, справиться с 2/3 задач — это круто.

Конечно, её "старшие братья" (более крупные модели на 32B или 72B) показывают точность выше (до 90%), но это и ожидаемо. 🏆

---

### Главные плюсы модели

*   **Понимает базу:** Отлично справляется с основами Python.
*   **Заточена под код:** Модель специально обучали на триллионах строк кода, и это чувствуется. 💻
*   **Есть куда расти:** Результат можно легко улучшить, если немного поиграться с настройками. 📈

## Уровень 3: Автоматический рефакторинг и "умные" правки

Этот уровень представляет собой качественный скачок от простого автодополнения кода к полноценному **интеллектуальному ассистенту**, который не просто предлагает следующий токен, а понимает семантику кода и проактивно предлагает улучшения. Это не «что написать дальше?», а «как написать это лучше?».

Системы этого уровня способны выполнять сложный рефакторинг, оптимизировать производительность и исправлять логические ошибки, действуя как опытный напарник-программист.

---

### Как это работает: разбор механики

В основе лежит комплексный, многоэтапный процесс, который превращает сырой код в проанализированную, понятную для модели структуру, и обратно.

### 1. Многоуровневый анализ структуры кода

Прежде чем вносить правки, система проводит глубокий анализ исходного кода, выходя далеко за рамки простого парсинга в синтаксическое дерево (AST). Она строит **семантическую модель** кодовой базы, которая включает:

*   **Синтаксическое дерево (AST):** Базовое представление структуры кода, показывающее иерархию операторов и выражений. Это «скелет» программы.
*   **Граф зависимостей:** Система строит карту связей между функциями, классами, переменными и модулями. Это позволяет понять, как изменение в одном месте (например, в функции `A`) повлияет на другое (на функцию `B`, которая вызывает `A`).
*   **Контекстная информация:** Собираются все метаданные:
    *   **Импорты:** Какие библиотеки и модули используются?
    *   **Область видимости (Scope):** Какие переменные доступны в данном участке кода?
    *   **Типы данных:** Явно или неявно определенные типы переменных и возвращаемых значений.
*   **Анализ паттернов и узких мест:** Система ищет распространенные анти-паттерны (например, неэффективные циклы, дублирование кода) и потенциальные точки для оптимизации (например, ресурсоемкие операции, которые можно распараллелить или кэшировать).

> **Цель этого этапа** — не просто «прочитать» код, а «понять» его на уровне архитектора, видя все взаимосвязи и потенциальные проблемы.

### 2. Интеллектуальное формирование промпта

Когда анализ завершен, система не отправляет в LLM весь файл с кодом. Она формирует **структурированный, обогащенный промпт**, который содержит всю собранную информацию в сжатом и понятном для модели виде.

Такой промпт может включать:

*   **Исходный код с аннотациями:** Критические участки кода помечаются специальными комментариями, указывающими на проблему (`// SLOW_PERFORMANCE`, `// CODE_DUPLICATION`).
*   **Упрощенное AST-представление:** Вместо полного дерева передаются только ключевые узлы, относящиеся к задаче рефакторинга.
*   **Контекст окружающего кода:** Краткое описание зависимостей, например: `Функция 'process_data' вызывается в 10 местах и работает с объектом типа 'User'`.
*   **Специфичные инструкции:** Четкая задача для модели, например: `«Рефакторинг функции 'get_user_list'. Замени неэффективный цикл на list comprehension. Убедись, что сохраняется исходная логика фильтрации»`.

> **Результат:** Модель получает не просто «стену текста», а техническое задание с полным контекстом, что на порядки повышает качество и релевантность генерации.


### 3. 🧪 Верификация и самокоррекция

Предложенное решение не сразу попадает к пользователю. Оно проходит через автоматизированный конвейер проверки:

1.  **Компиляция / Интерпретация:** Система проверяет, что сгенерированный код синтаксически корректен и вообще запускается.
2.  **Автоматическое тестирование:** Запускаются существующие юнит-тесты, относящиеся к измененному коду. В идеале, система может сама сгенерировать временные тесты для проверки краевых случаев.
3.  **Обратная связь и обучение (Self-Correction):**
    *   **Если тесты провалены ✅ ➡️ ❌:** Результат с информацией об ошибке отправляется обратно в мульти-агентную систему. `Reviewer Agent` получает сообщение: `«Тест 'test_empty_list' провален с ошибкой 'IndexError'»`.
    *   **Reinforcement Learning:** Этот цикл обратной связи (генерация -> тест -> провал -> новая генерация) позволяет системе обучаться на своих ошибках, постепенно улучшая качество предложений.

> **Итог:** Пользователь получает не просто сгенерированный код, а решение, которое уже прошло несколько этапов проверки, компиляции и тестирования, что значительно повышает его надежность.

In [228]:
import ast
import json
import inspect
from typing import Dict, List, Any, Optional
from dataclasses import dataclass

In [229]:
@dataclass
class RefactoringContext:
    """Контекст для рефакторинга с метаданными"""
    source_code: str
    ast_tree: ast.AST
    function_name: str
    complexity_score: int
    dependencies: List[str]
    suggested_patterns: List[str]

class AdvancedCodeAnalyzer:
    """Продвинутый анализатор кода для формирования промптов"""
    
    def __init__(self):
        self.complexity_threshold = 10
    
    def extract_code_metrics(self, source_code: str) -> Dict[str, Any]:
        """Извлекает метрики сложности кода"""
        tree = ast.parse(source_code)
        
        metrics = {
            'cyclomatic_complexity': self._calculate_complexity(tree),
            'lines_of_code': len(source_code.split('\n')),
            'nested_loops': self._count_nested_loops(tree),
            'function_calls': self._extract_function_calls(tree),
            'variables': self._extract_variables(tree)
        }
        return metrics
    
    def _calculate_complexity(self, tree: ast.AST) -> int:
        """Упрощенная оценка цикломатической сложности"""
        complexity = 1
        for node in ast.walk(tree):
            if isinstance(node, (ast.If, ast.While, ast.For, ast.Try)):
                complexity += 1
        return complexity
    
    def _count_nested_loops(self, tree: ast.AST) -> int:
        """Подсчет вложенных циклов"""
        max_depth = 0
        current_depth = 0
        
        for node in ast.walk(tree):
            if isinstance(node, (ast.For, ast.While)):
                current_depth += 1
                max_depth = max(max_depth, current_depth)
            # Упрощенная логика - в реальности нужен более сложный обход
        return max_depth
    
    def _extract_function_calls(self, tree: ast.AST) -> List[str]:
        """Извлекает вызовы функций"""
        calls = []
        for node in ast.walk(tree):
            if isinstance(node, ast.Call):
                if isinstance(node.func, ast.Name):
                    calls.append(node.func.id)
        return calls
    
    def _extract_variables(self, tree: ast.AST) -> List[str]:
        """Извлекает имена переменных"""
        variables = []
        for node in ast.walk(tree):
            if isinstance(node, ast.Name) and isinstance(node.ctx, ast.Store):
                variables.append(node.id)
        return variables
    
    def create_semantic_ast_summary(self, tree: ast.AST) -> Dict[str, Any]:
        """Создает семантическое резюме AST для промпта"""
        summary = {
            'structure': 'function_definition',
            'control_flow': [],
            'data_operations': [],
            'optimization_opportunities': []
        }
        
        for node in ast.walk(tree):
            if isinstance(node, ast.For):
                summary['control_flow'].append({
                    'type': 'for_loop',
                    'target': node.target.id if isinstance(node.target, ast.Name) else 'complex',
                    'iter': node.iter.id if isinstance(node.iter, ast.Name) else 'expression'
                })
            elif isinstance(node, ast.If):
                summary['control_flow'].append({
                    'type': 'conditional',
                    'test_type': type(node.test).__name__
                })
            elif isinstance(node, ast.Assign):
                summary['data_operations'].append({
                    'type': 'assignment',
                    'targets': [t.id for t in node.targets if isinstance(t, ast.Name)]
                })
            elif isinstance(node, ast.Call) and isinstance(node.func, ast.Attribute):
                if node.func.attr == 'append':
                    summary['optimization_opportunities'].append('list_comprehension_candidate')
        
        return summary

def create_advanced_refactoring_prompt(context: RefactoringContext) -> str:
    """Создает продвинутый промпт для рефакторинга"""
    
    analyzer = AdvancedCodeAnalyzer()
    metrics = analyzer.extract_code_metrics(context.source_code)
    semantic_summary = analyzer.create_semantic_ast_summary(context.ast_tree)
    
    prompt = f"""
        # Code Refactoring Task
        
        ## Context
        - **Function**: `{context.function_name}`
        - **Complexity Score**: {context.complexity_score}/10
        - **Current Issues**: {', '.join(context.suggested_patterns)}
        
        ## Source Code Analysis
        
        {context.source_code.strip()}
        
        
        ## Code Metrics
        - **Lines of Code**: {metrics['lines_of_code']}
        - **Cyclomatic Complexity**: {metrics['cyclomatic_complexity']}
        - **Nested Loops**: {metrics['nested_loops']}
        - **Function Calls**: {', '.join(metrics['function_calls']) if metrics['function_calls'] else 'None'}
        - **Variables**: {', '.join(metrics['variables'])}
        
        ## Semantic Structure Analysis
        
        {json.dumps(semantic_summary, indent=2)}
        
        
        ## Refactoring Requirements
        1. **Primary Goal**: Convert imperative loop to functional approach
        2. **Performance**: Optimize for readability and efficiency
        3. **Style**: Follow PEP 8 and Python idioms
        4. **Constraints**: 
           - Preserve original functionality exactly
           - Maintain type hints compatibility
           - Keep function signature unchanged
        
        ## Expected Improvements
        - Replace manual list building with list comprehension
        - Eliminate unnecessary intermediate variables
        - Improve code readability and maintainability
        
        ## Output Format
        Please provide:
        1. **Refactored code** with comments explaining changes
        2. **Performance analysis** comparing old vs new approach
        3. **Risk assessment** for potential edge cases
        
        Generate the refactored version:
        """
    
    return prompt

In [230]:
def code_to_refactor(items):
    """Фильтрует четные числа и возводит их в квадрат"""
    new_list = []
    for i in items:
        if i % 2 == 0:
            new_list.append(i * i)
    return new_list

In [231]:
# Создаем контекст рефакторинга
source_code = inspect.getsource(code_to_refactor)
tree = ast.parse(source_code)

context = RefactoringContext(
    source_code=source_code,
    ast_tree=tree,
    function_name='code_to_refactor',
    complexity_score=3,
    dependencies=['builtins'],
    suggested_patterns=['list_comprehension_candidate', 'functional_approach']
)

# Генерируем улучшенный промпт
advanced_prompt = create_advanced_refactoring_prompt(context)

In [232]:
print("=== УЛУЧШЕННЫЙ ПРОМПТ ===")
print(advanced_prompt)

=== УЛУЧШЕННЫЙ ПРОМПТ ===

        # Code Refactoring Task

        ## Context
        - **Function**: `code_to_refactor`
        - **Complexity Score**: 3/10
        - **Current Issues**: list_comprehension_candidate, functional_approach

        ## Source Code Analysis

        def code_to_refactor(items):
    """Фильтрует четные числа и возводит их в квадрат"""
    new_list = []
    for i in items:
        if i % 2 == 0:
            new_list.append(i * i)
    return new_list


        ## Code Metrics
        - **Lines of Code**: 8
        - **Cyclomatic Complexity**: 3
        - **Nested Loops**: 1
        - **Function Calls**: None
        - **Variables**: new_list, i

        ## Semantic Structure Analysis

        {
  "structure": "function_definition",
  "control_flow": [
    {
      "type": "for_loop",
      "target": "i",
      "iter": "items"
    },
    {
      "type": "conditional",
      "test_type": "Compare"
    }
  ],
  "data_operations": [
    {
      "type": "assignmen

In [233]:
# Пример ожидаемого ответа от LLM
expected_refactored = """
def code_to_refactor(items):
    \"\"\"Фильтрует четные числа и возводит их в квадрат
    
    Refactored to use list comprehension for better performance and readability.
    \"\"\"
    return [i * i for i in items if i % 2 == 0]

# Performance Analysis:
# - Old: O(n) with explicit loop and list.append() calls
# - New: O(n) with optimized list comprehension (faster in CPython)
# - Memory: Reduced intermediate variable usage
# - Readability: More Pythonic and concise

# Risk Assessment: LOW
# - Functionality preserved exactly
# - No edge case changes
# - Type compatibility maintained
"""

print("\n=== ОЖИДАЕМЫЙ РЕЗУЛЬТАТ ===")
print(expected_refactored)


=== ОЖИДАЕМЫЙ РЕЗУЛЬТАТ ===

def code_to_refactor(items):
    """Фильтрует четные числа и возводит их в квадрат

    Refactored to use list comprehension for better performance and readability.
    """
    return [i * i for i in items if i % 2 == 0]

# Performance Analysis:
# - Old: O(n) with explicit loop and list.append() calls
# - New: O(n) with optimized list comprehension (faster in CPython)
# - Memory: Reduced intermediate variable usage
# - Readability: More Pythonic and concise

# Risk Assessment: LOW
# - Functionality preserved exactly
# - No edge case changes
# - Type compatibility maintained



In [240]:
response = ollama.Client().generate(model="qwen2.5-coder:14b", prompt=advanced_prompt)

In [239]:
print(response.response)

```python
def code_to_refactor(items) -> list[int]:
    """Фильтрует четные числа и возводит их в квадрат"""
    # Refactored using list comprehension to improve readability and efficiency
    return [i * i for i in items if i % 2 == 0]
```

### Performance Analysis

#### Old Approach:
- **Time Complexity**: O(n)
- **Space Complexity**: O(n) due to the creation of `new_list`.
- **Execution Steps**:
  - Iterates over each element in `items` once.
  - Checks if the element is even.
  - If even, appends its square to `new_list`.

#### New Approach (List Comprehension):
- **Time Complexity**: O(n)
- **Space Complexity**: O(n) due to the creation of a new list.
- **Execution Steps**:
  - Iterates over each element in `items` once.
  - Checks if the element is even and squares it in one expression.

### Risk Assessment

1. **Type Safety**: The function signature specifies that the input should be a list of integers (`list[int]`). List comprehension maintains this type safety, as it processes

## Уровень 4: Понимание всего проекта (Full Codebase Awareness)


Это принципиально другой подход, основанный на **Retrieval-Augmented Generation (RAG)**.
Переходим на самый высокий уровень интеллектуальных ассистентов. Здесь система перестает быть просто анализатором одного файла и становится **полноценным архитектором**, который "знает" весь ваш проект целиком. Она способна отвечать на вопросы, для которых контекст разбросан по десяткам файлов, находить неочевидные зависимости и предлагать глобальные рефакторинги.

**Ключевая идея:** Вместо того чтобы пытаться "засунуть" весь проект в ограниченное окно LLM, мы используем умный поиск, чтобы дать модели только ту информацию, которая нужна для ответа здесь и сейчас.

---

## Как это работает: Retrieval-Augmented Generation (RAG)

В основе этого подхода лежит технология **RAG** — генерация, обогащенная поиском. Процесс состоит из двух фаз: индексации (один раз) и поиска (при каждом запросе).

###  Фаза 1: Индексация проекта (происходит заранее, в фоне)

1.  **📚 Сканирование и Чанкинг:** Ассистент сканирует все файлы в проекте. Он не просто читает текст, а использует **семантический чанкинг**, разбивая код на логические блоки: функции, классы, методы. *(Подробнее о методах чанкинга — ниже)*.

2.  **🧠 Создание Эмбеддингов:** Каждый такой блок (чанк) пропускается через специальную модель-эмбеддер, которая превращает его в **вектор** — числовое представление семантики кода. В этом векторе закодирована информация о том, *что* делает код, а не просто из каких символов он состоит.

3.  **💾 Сохранение в Векторную Базу:** Все полученные векторы (эмбеддинги) вместе с исходным кодом сохраняются в локальную векторную базу данных (например, `FAISS`, `ChromaDB`). Эта база данных оптимизирована для сверхбыстрого поиска векторов по "смысловой близости".

### Фаза 2: Работа с запросом (происходит в реальном времени)

4.  **❓ Запрос пользователя:** Вы задаете вопрос или выделяете код, например: *"Где используется функция `getUserPermissions` и как она обрабатывает гостевых пользователей?"*

5.  **🔍 Векторный поиск:** Система превращает ваш текстовый запрос в такой же вектор-эмбеддинг и ищет в базе данных наиболее близкие по смыслу векторы кода. Результатом будут те самые чанки (функции, классы) из вашего проекта, которые наиболее релевантны вопросу.

6.  **✨ Обогащение промпта:** Найденные фрагменты кода добавляются в промпт для основной LLM в качестве дополнительного контекста. Финальный промпт для модели выглядит примерно так:

> **Системный промпт:** "Ты — эксперт-помощник по программированию. Ответь на вопрос пользователя, используя предоставленный контекст из его кодовой базы."
> **Контекст (найденные чанки):**
> ```python
> # file: services/permissions.py
> def getUserPermissions(user_id):
>   # ... (код функции)
> ```
> ```python
> # file: controllers/api.py
> permissions = getUserPermissions(current_user.id)
> if not permissions:
>   # ... (обработка гостя)
> ```
> **Вопрос пользователя:** "Где используется функция `getUserPermissions` и как она обрабатывает гостевых пользователей?"

**Результат:** LLM получает всю необходимую информацию для точного и контекстуально-верного ответа, даже если нужный код разбросан по разным файлам.


## Модели эмбеддингов для кода

Ключ к успеху RAG — это качество эмбеддингов. Модели, которые переводят код в векторы, — это настоящее произведение искусства.

### Почему они особенные?

Обычные модели для эмбеддинга текста (вроде тех, что используются в поиске Google) плохо понимают код. Специализированные code-embedding модели обучаются улавливать:
*   **Синтаксис:** Важность скобок, отступов, ключевых слов.
*   **Логику выполнения:** Что `for item in items` — это цикл.
*   **Зависимости:** Что изменение одной переменной повлияет на другую.
*   **Функциональность:** Что две функции, написанные по-разному, могут делать одно и то же.

### Топовые модели (2024-2025)

| Модель |Ключевые особенности | Область применения |
| :--- | :--- | :--- |
| **[VoyageCode-3](https://huggingface.co/voyageai/voyage-code-3)**  | 🏆 Лидер по качеству семантического поиска кода. | RAG для кода, поиск по смыслу. |
| **[CodeXEmbed](https://huggingface.co/papers/2411.12644)** | 🌐 Open-source, поддерживает 12+ языков. | Универсальный поиск (код-текст, текст-код). |
| **[Qodo-Embed-1](https://huggingface.co/Qodo/Qodo-Embed-1-7B)** | 🚀 SOTA на бенчмарке CoIR, использует синтетические данные. | Поиск кода (code retrieval). |
| **[Jina Code V2](https://huggingface.co/jinaai/jina-embeddings-v2-base-code)** | ⚙️ Оптимизирован под паттерны использования API. | Автодополнение, анализ репозиториев. |
| **OpenAI `text-embedding-3-large`** |  универсальная модель с хорошей поддержкой кода. | Гибридные задачи (текст + код). |

### Как их обучают? 

1.  **Контрастивное обучение с AST-трансформациями**
    Это самый продвинутый метод. Модель учится отличать семантически одинаковый код от разного.
    *   **Anchor (Якорь):** Исходная функция.
    *   **Positive (Позитивный пример):** Та же функция, но с измененными именами переменных, другим стилем форматирования (семантика сохранена).
    *   **Negative (Негативный пример):** Совершенно другие функции.

    Модель штрафуют, если вектор "якоря" далек от "позитивного" или близок к "негативному".

    ```python
    # Anchor:
    def original_function(items):
        result = []
        for item in items:
            result.append(item * 2)
        return result

    # Positive Sample (семантически эквивалентен):
    def transformed_function(data):
        output = [element * 2 for element in data]
        return output
    ```

2.  **Мульти-задачное обучение (Multi-task Training)**
    Модель (`CodeXEmbed`) обучается сразу нескольким задачам, которые сводятся к поиску:
    *   **Код ➡️ Текст:** Найди документацию для этой функции.
    *   **Текст ➡️ Код:** Найди функцию по этому описанию.
    *   **Код ➡️ Код:** Найди похожие по функциональности функции.


<div style="text-align: center;">
  <img src="docs/code_emb_training.png" alt="alt text" style="width:400px;">
</div>

3.  **Генерация синтетических данных**
    Подход (`Qodo-Embed-1`). Большая LLM (например, GPT-4) используется для генерации тысяч пар "описание на естественном языке ↔️ фрагмент кода". Это позволяет модели лучше понимать человеческие запросы о коде.

## Продвинутые подходы к чанкингу

"Как порезать слона?" — главный вопрос RAG. Простое разбиение кода по 500 символов не работает, так как разрывает логические блоки. Поэтому используются умные стратегии.

### 1. Семантический чанкинг (Content-Aware Chunking)

Система использует парсеры кода (как в компиляторах), чтобы разбивать файлы не по тексту, а по структуре:
*   **Уровень функций/методов:** Каждая функция — отдельный чанк.
*   **Уровень классов:** Весь класс с его методами — один чанк.
*   **Уровень модулей:** Группировка связанных импортов и глобальных переменных.

### 2. Иерархический чанкинг

Создается многоуровневое представление кода для гибкого поиска.
*   **Макро-уровень (1000-2000 токенов):** Целые файлы или большие классы. Дают общий контекст.
*   **Микро-уровень (200-500 токенов):** Отдельные функции или методы. Дают конкретную реализацию.
*   **Нано-уровень (50-100 токенов):** Отдельные выражения или логические блоки. Для очень точечных вопросов.

### 3. Стратегии "перекрытия" (Overlap)

Чтобы чанки не теряли контекст на границах, они создаются с перекрытием, которое захватывает важную информацию из соседей.
*   **Синтаксическое перекрытие:** В чанк включается объявление переменных из предыдущего блока.
*   **Контекстное перекрытие:** В чанк всегда включаются docstring'и и комментарии, относящиеся к нему.
*   **Зависимостное перекрытие:** В чанк с функцией добавляются ключевые импорты из начала файла.

In [227]:
# проверим на практическом примере

In [219]:
import langchain, langchain_community
import os, ast, re
from typing import List, Dict, Any, Optional
import tiktoken
from dataclasses import dataclass
from langchain.text_splitter import Language, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

In [220]:
print("tiktoken", tiktoken.__version__)
print("langchain", langchain.__version__)
print("langchain_community", langchain_community.__version__)

tiktoken 0.9.0
langchain 0.3.25
langchain_community 0.3.25


In [ ]:
@dataclass
class CodeChunk:
    """Enhanced code chunk with semantic metadata"""
    content: str
    file_path: str
    function_name: Optional[str] = None
    class_name: Optional[str] = None
    line_start: int = 0
    line_end: int = 0
    dependencies: List[str] = None
    chunk_type: str = "general"  # function, class, import, comment

class AdvancedCodeChunker:
    """Production-ready code chunking with multiple strategies"""
    
    def __init__(self, model_name: str = "BAAI/bge-code-base"):
        # Используем специализированную модель для кода вместо general-purpose
        self.embeddings = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={'device': 'cpu'},
            encode_kwargs={'normalize_embeddings': True}
        )
        
        # Токенизатор для точного подсчета токенов
        self.encoding = tiktoken.get_encoding("cl100k_base")
        
        # Различные стратегии chunking
        self.strategies = {
            'semantic': self._semantic_chunking,
            'hybrid': self._hybrid_chunking,
            'function_based': self._function_based_chunking
        }
    
    def _count_tokens(self, text: str) -> int:
        """Точный подсчет токенов"""
        return len(self.encoding.encode(text))
    
    def _extract_functions_and_classes(self, code: str, file_path: str) -> List[CodeChunk]:
        """Извлекает функции и классы как отдельные чанки"""
        chunks = []
        try:
            tree = ast.parse(code)
            lines = code.split('\n')
            
            for node in ast.walk(tree):
                if isinstance(node, (ast.FunctionDef, ast.AsyncFunctionDef)):
                    start_line = node.lineno - 1
                    end_line = node.end_lineno if hasattr(node, 'end_lineno') else start_line + 10
                    
                    func_code = '\n'.join(lines[start_line:end_line])
                    
                    # Извлекаем зависимости (вызовы других функций)
                    dependencies = []
                    for call_node in ast.walk(node):
                        if isinstance(call_node, ast.Call) and isinstance(call_node.func, ast.Name):
                            dependencies.append(call_node.func.id)
                    
                    chunks.append(CodeChunk(
                        content=func_code,
                        file_path=file_path,
                        function_name=node.name,
                        line_start=start_line,
                        line_end=end_line,
                        dependencies=dependencies,
                        chunk_type="function"
                    ))
                    
                elif isinstance(node, ast.ClassDef):
                    start_line = node.lineno - 1
                    end_line = node.end_lineno if hasattr(node, 'end_lineno') else start_line + 20
                    
                    class_code = '\n'.join(lines[start_line:end_line])
                    chunks.append(CodeChunk(
                        content=class_code,
                        file_path=file_path,
                        class_name=node.name,
                        line_start=start_line,
                        line_end=end_line,
                        chunk_type="class"
                    ))
                    
        except SyntaxError:
            # Fallback для некорректного кода
            return self._fallback_chunking(code, file_path)
            
        return chunks
    
    def _semantic_chunking(self, code: str, file_path: str, target_size: int = 400) -> List[CodeChunk]:
        """Семантический chunking с учетом структуры кода"""
        chunks = []
        
        # Сначала пробуем извлечь функции и классы
        semantic_chunks = self._extract_functions_and_classes(code, file_path)
        
        for chunk in semantic_chunks:
            token_count = self._count_tokens(chunk.content)
            
            if token_count <= target_size:
                chunks.append(chunk)
            else:
                # Слишком большие функции разбиваем дальше
                sub_chunks = self._split_large_function(chunk, target_size)
                chunks.extend(sub_chunks)
        
        # Обрабатываем оставшийся код (импорты, глобальные переменные)
        remaining_code = self._extract_remaining_code(code, semantic_chunks)
        if remaining_code.strip():
            chunks.append(CodeChunk(
                content=remaining_code,
                file_path=file_path,
                chunk_type="imports_globals"
            ))
            
        return chunks
    
    def _hybrid_chunking(self, code: str, file_path: str, target_size: int = 300) -> List[CodeChunk]:
        """Гибридный подход: semantic + overlap-based"""
        semantic_chunks = self._semantic_chunking(code, file_path, target_size)
        
        # Добавляем overlap между связанными чанками
        enhanced_chunks = []
        
        for i, chunk in enumerate(semantic_chunks):
            enhanced_content = chunk.content
            
            # Добавляем контекст из предыдущего чанка
            if i > 0 and semantic_chunks[i-1].chunk_type in ["function", "class"]:
                prev_signature = self._extract_signature(semantic_chunks[i-1].content)
                if prev_signature:
                    enhanced_content = f"# Previous context:\n{prev_signature}\n\n{enhanced_content}"
            
            # Добавляем импорты если их нет
            if chunk.chunk_type == "function" and not self._has_imports(chunk.content):
                imports = self._extract_imports(code)
                if imports:
                    enhanced_content = f"{imports}\n\n{enhanced_content}"
            
            enhanced_chunks.append(CodeChunk(
                content=enhanced_content,
                file_path=chunk.file_path,
                function_name=chunk.function_name,
                class_name=chunk.class_name,
                line_start=chunk.line_start,
                line_end=chunk.line_end,
                dependencies=chunk.dependencies,
                chunk_type=chunk.chunk_type
            ))
            
        return enhanced_chunks
    
    def _extract_signature(self, code: str) -> Optional[str]:
        """Извлекает сигнатуру функции/класса"""
        lines = code.strip().split('\n')
        for line in lines:
            if line.strip().startswith(('def ', 'class ', 'async def ')):
                return line.strip()
        return None
    
    def _has_imports(self, code: str) -> bool:
        """Проверяет наличие импортов в коде"""
        return bool(re.search(r'^\s*(import|from)\s+', code, re.MULTILINE))
    
    def _extract_imports(self, code: str) -> str:
        """Извлекает все импорты из кода"""
        lines = code.split('\n')
        imports = []
        for line in lines:
            if re.match(r'^\s*(import|from)\s+', line):
                imports.append(line)
        return '\n'.join(imports)
    
    def process_codebase(self, codebase: Dict[str, str], 
                        strategy: str = "hybrid", 
                        target_size: int = 300) -> List[Document]:
        """Обрабатывает всю кодовую базу"""
        
        all_chunks = []
        chunking_func = self.strategies.get(strategy, self._hybrid_chunking)
        
        for file_path, code in codebase.items():
            file_chunks = chunking_func(code, file_path, target_size)
            
            # Конвертируем в LangChain Documents
            for chunk in file_chunks:
                doc = Document(
                    page_content=chunk.content,
                    metadata={
                        'file': chunk.file_path,
                        'function_name': chunk.function_name,
                        'class_name': chunk.class_name,
                        'chunk_type': chunk.chunk_type,
                        'line_start': chunk.line_start,
                        'line_end': chunk.line_end,
                        'dependencies': chunk.dependencies or [],
                        'token_count': self._count_tokens(chunk.content)
                    }
                )
                all_chunks.append(doc)
        
        return all_chunks
    
    # Helper methods
    def _split_large_function(self, chunk: CodeChunk, target_size: int) -> List[CodeChunk]:
        """Разбивает большие функции на меньшие части"""
        # Упрощенная версия - можно расширить
        splitter = RecursiveCharacterTextSplitter.from_language(
            language=Language.PYTHON,
            chunk_size=target_size,
            chunk_overlap=50,
            length_function=self._count_tokens
        )
        
        docs = splitter.create_documents([chunk.content])
        sub_chunks = []
        
        for i, doc in enumerate(docs):
            sub_chunks.append(CodeChunk(
                content=doc.page_content,
                file_path=chunk.file_path,
                function_name=f"{chunk.function_name}_part_{i+1}",
                chunk_type="function_part",
                dependencies=chunk.dependencies
            ))
        
        return sub_chunks
    
    def _extract_remaining_code(self, code: str, processed_chunks: List[CodeChunk]) -> str:
        """Извлекает код, не попавший в функции/классы"""
        lines = code.split('\n')
        processed_lines = set()
        
        for chunk in processed_chunks:
            for line_num in range(chunk.line_start, chunk.line_end + 1):
                if line_num < len(lines):
                    processed_lines.add(line_num)
        
        remaining_lines = []
        for i, line in enumerate(lines):
            if i not in processed_lines and line.strip():
                remaining_lines.append(line)
        
        return '\n'.join(remaining_lines)
    
    def _fallback_chunking(self, code: str, file_path: str) -> List[CodeChunk]:
        """Fallback chunking для проблематичного кода"""
        splitter = RecursiveCharacterTextSplitter.from_language(
            language=Language.PYTHON,
            chunk_size=300,
            chunk_overlap=30
        )
        
        docs = splitter.create_documents([code])
        return [CodeChunk(content=doc.page_content, file_path=file_path) for doc in docs]
    
    def _function_based_chunking(self, code: str, file_path: str, target_size: int = 400) -> List[CodeChunk]:
        """Простой function-based chunking"""
        return self._extract_functions_and_classes(code, file_path)

In [ ]:
# это наша кодовая база
codebase = {
    "user_service.py": """
        import logging
        from typing import Optional, List
        from database import get_database_connection, Database
        
        logger = logging.getLogger(__name__)
        
        class UserAPI:
            \"\"\"API for user operations\"\"\"
            
            def __init__(self, db_config: dict):
                self.db_config = db_config
                self.cache = {}
            
            def get_user(self, user_id: int) -> Optional[dict]:
                \"\"\"Retrieves user from the database\"\"\"
                if user_id in self.cache:
                    logger.info(f"Cache hit for user {user_id}")
                    return self.cache[user_id]
                    
                db = get_database_connection()
                user_data = db.query(f'SELECT * FROM users WHERE id = {user_id}')
                
                if user_data:
                    self.cache[user_id] = user_data
                    logger.info(f"User {user_id} loaded from database")
                
                return user_data
            
            def create_user(self, username: str, email: str) -> dict:
                \"\"\"Creates a new user\"\"\"
                db = get_database_connection()
                query = f"INSERT INTO users (username, email) VALUES ('{username}', '{email}')"
                result = db.execute(query)
                
                new_user = {
                    'id': result.lastrowid,
                    'username': username, 
                    'email': email
                }
                
                # Очищаем кэш
                self.cache.clear()
                logger.info(f"Created user: {username}")
                return new_user
    """,
    "database.py": """
        import os
        import sqlite3
        from typing import Optional, Any
        import logging
        
        logger = logging.getLogger(__name__)
        
        class Database:
            \"\"\"Database connection wrapper\"\"\"
            
            def __init__(self, db_path: str = "app.db"):
                self.db_path = db_path
                self.connection = None
                self._connect()
            
            def _connect(self):
                \"\"\"Establish database connection\"\"\"
                try:
                    self.connection = sqlite3.connect(self.db_path)
                    self.connection.row_factory = sqlite3.Row
                    logger.info(f"Connected to database: {self.db_path}")
                except Exception as e:
                    logger.error(f"Database connection failed: {e}")
                    raise
            
            def query(self, sql: str) -> Optional[dict]:
                \"\"\"Execute SELECT query\"\"\"
                logger.debug(f"Executing query: {sql}")
                try:
                    cursor = self.connection.cursor()
                    cursor.execute(sql)
                    result = cursor.fetchone()
                    return dict(result) if result else None
                except Exception as e:
                    logger.error(f"Query failed: {e}")
                    return None
            
            def execute(self, sql: str) -> Any:
                \"\"\"Execute INSERT/UPDATE/DELETE query\"\"\"
                logger.debug(f"Executing: {sql}")
                try:
                    cursor = self.connection.cursor()
                    cursor.execute(sql)
                    self.connection.commit()
                    return cursor
                except Exception as e:
                    logger.error(f"Execute failed: {e}")
                    self.connection.rollback()
                    raise
        
        # Singleton instance
        _db_instance = None
        
        def get_database_connection() -> Database:
            \"\"\"Returns a database connection singleton\"\"\"
            global _db_instance
            if _db_instance is None:
                db_path = os.getenv('DB_PATH', 'default.db')
                _db_instance = Database(db_path)
            return _db_instance
    """,
    "config.py": """
        import os
        from dataclasses import dataclass
        
        @dataclass
        class AppConfig:
            debug: bool = False
            db_path: str = "app.db"
            log_level: str = "INFO"
            cache_size: int = 1000
        
        def load_config() -> AppConfig:
            return AppConfig(
                debug=os.getenv('DEBUG', 'False').lower() == 'true',
                db_path=os.getenv('DB_PATH', 'app.db'),
                log_level=os.getenv('LOG_LEVEL', 'INFO'),
                cache_size=int(os.getenv('CACHE_SIZE', '1000'))
            )
    """
}

In [221]:
# будем использовать эту модель эмбедингов
chunker = AdvancedCodeChunker(model_name="BAAI/bge-code-v1")

# Обработка с разными стратегиями
strategies = ["semantic", "hybrid", "function_based"]

for strategy in strategies:
    print(f"\n--- Стратегия: {strategy.upper()} ---")
    
    # Обрабатываем кодовую базу
    docs = chunker.process_codebase(codebase, strategy=strategy, target_size=350)
    
    print(f"Создано {len(docs)} чанков")
    
    # Показываем статистику по типам чанков
    chunk_types = {}
    token_stats = []
    
    for doc in docs:
        chunk_type = doc.metadata['chunk_type']
        chunk_types[chunk_type] = chunk_types.get(chunk_type, 0) + 1
        token_stats.append(doc.metadata['token_count'])
    
    print("Распределение по типам:")
    for chunk_type, count in chunk_types.items():
        print(f"  {chunk_type}: {count}")
    
    if token_stats:
        avg_tokens = sum(token_stats) / len(token_stats)
        print(f"Средний размер чанка: {avg_tokens:.1f} токенов")
    
    # Создаем vectorstore и тестируем поиск
    try:
        vectorstore = FAISS.from_documents(docs, chunker.embeddings)
        retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
        
        # Тестовые запросы
        test_queries = [
            "Как создается подключение к базе данных?",
            "Где используется кэширование пользователей?", 
            "Какие есть функции для работы с пользователями?"
        ]
        
        for query in test_queries:
            print(f"\nЗапрос: {query}")
            relevant_docs = retriever.invoke(query)
            
            print("Найденные чанки:")
            for i, doc in enumerate(relevant_docs, 1):
                metadata = doc.metadata
                print(f"  {i}. {metadata['file']} ({metadata['chunk_type']})")
                if metadata.get('function_name'):
                    print(f"      Функция: {metadata['function_name']}")
                print(f"      Размер: {metadata['token_count']} токенов")
                print(f"      Превью: {doc.page_content[:100]}...")
                
    except Exception as e:
        print(f"Ошибка при создании vectorstore: {e}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


--- Стратегия: SEMANTIC ---
Создано 15 чанков
Распределение по типам:
  class: 3
  function: 9
  imports_globals: 3
Средний размер чанка: 87.3 токенов

Запрос: Как создается подключение к базе данных?
Найденные чанки:
  1. user_service.py (imports_globals)
      Размер: 26 токенов
      Превью: import logging
from typing import Optional, List
from database import get_database_connection, Datab...
  2. config.py (function)
      Функция: load_config
      Размер: 68 токенов
      Превью: def load_config() -> AppConfig:
    return AppConfig(
        debug=os.getenv('DEBUG', 'False').lowe...
  3. database.py (class)
      Размер: 278 токенов
      Превью: class Database:
    """Database connection wrapper"""

    def __init__(self, db_path: str = "app.db...

Запрос: Где используется кэширование пользователей?
Найденные чанки:
  1. config.py (class)
      Размер: 38 токенов
      Превью: class AppConfig:
    debug: bool = False
    db_path: str = "app.db"
    log_level: str = "INFO"
   ..

In [222]:
# Используем стратегию hybrid
final_docs = chunker.process_codebase(codebase, strategy="hybrid", target_size=300)
vectorstore = FAISS.from_documents(final_docs, chunker.embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

user_question = "Как функция get_user получает доступ к базе данных и использует кэширование?"
relevant_docs = retriever.invoke(user_question)

# Формируем обогащенный контекст
enriched_context = []
for doc in relevant_docs:
    meta = doc.metadata
    context_header = f"--- {meta['file']}"
    
    if meta.get('function_name'):
        context_header += f" | Функция: {meta['function_name']}"
    if meta.get('class_name'):
        context_header += f" | Класс: {meta['class_name']}"
    if meta.get('dependencies'):
        context_header += f" | Зависимости: {', '.join(meta['dependencies'][:3])}"
    
    context_header += f" | Токены: {meta['token_count']} ---"
    
    enriched_context.append(f"{context_header}\n{doc.page_content}")

final_context = "\n\n".join(enriched_context)

In [223]:
advanced_prompt = f"""
    # Code Analysis Task
    ## User Question
    {user_question}
    
    ## Retrieved Code Context
    {final_context}
    
    ## Analysis Instructions
    1. **Trace the execution flow** между найденными фрагментами кода
    2. **Identify patterns** использования баз данных и кэширования  
    3. **Explain dependencies** между компонентами
    4. **Provide concrete examples** из найденного кода
    
    ## Expected Output Format
    - **Основной ответ**: прямой ответ на вопрос
    - **Code flow**: пошаговое объяснение выполнения
    - **Key components**: список ключевых функций/классов
    - **Potential improvements**: рекомендации по улучшению
    Ответ:
"""

In [210]:
response = ollama.Client().generate(model="qwen2.5-coder:14b", prompt=advanced_prompt)

In [224]:
print(user_question)

Как функция get_user получает доступ к базе данных и использует кэширование?


In [225]:
print(response.response)

## Основной ответ
Функция `get_user` получает доступ к базе данных, используя функцию `get_database_connection`, которая возвращает объект базы данных. Если запрошенный пользователь уже находится в кэше (`self.cache`), функция возвращает данные из кэша, что называется "кэш-хитом". Если пользователя нет в кэше, функция выполняет SQL-запрос для выборки пользователя из базы данных. После получения данных пользователь добавляется в кэш, чтобы последующие запросы этого пользователя могли быть обработаны быстрее.

## Code flow
1. Функция `get_user` принимает аргумент `user_id`.
2. Проверяется наличие `user_id` в кэше (`self.cache`).
3. Если `user_id` найден в кэше:
   - Возвращаются данные из кэша.
4. Если `user_id` не найден в кэше:
   - Вызывается функция `get_database_connection`, которая возвращает объект базы данных.
   - Функция выполняет SQL-запрос для выборки пользователя из базы данных (`db.query(f'SELECT * FROM users WHERE id = {user_id}')`).
5. Если пользователь найден в базе данн

# Уровень 5: «Агенты» и автономные задачи  

**Как это работает?**

Это циклический процесс, часто описываемый фреймворком **ReAct (Reason + Act)**.

- **Планирование (Reason)**: LLM получает вашу цель и разбивает ее на последовательность конкретных шагов: `"1. Найти файл с роутами. 2. Добавить новый роут. 3. Создать файл контроллера. 4. Написать логику в контроллере..."`.
- **Использование инструментов (Act)**: У "агента" есть доступ к набору инструментов — API для взаимодействия со средой: `readFile(path)`, `writeFile(path, content)`, `findSymbol(name)`, `runTerminalCommand(command)`.
- **Исполнение и наблюдение**: Агент выполняет первый шаг плана, используя нужный инструмент (например, `readFile('src/routes.js')`). Он получает результат (содержимое файла или ошибку) и анализирует его.
- **Повторение цикла**: На основе результата агент корректирует свой план и переходит к следующему шагу. Этот цикл `"Подумал -> Сделал -> Посмотрел на результат"` продолжается до тех пор, пока задача не будет выполнена или агент не столкнется с проблемой, для решения которой ему нужна помощь человека.

<div style="text-align: center;">
  <img src="docs/react_1.png" alt="alt text" style="width:400px;">
</div>


## Архитектурная карта code-агента

### Базовые слои

| Слой | Назначение | Ключевые компоненты |
|------|------------|---------------------|
| Интерфейс цели | Приём «high-level» запроса пользователя. | Form / API-endpoint, CLI, чат. |
| Планировщик (*Reason*) | Декомпозирует цель в план действий. | Chain-of-Thought, Tree-of-Thought, Task-Tree генератор |
| Исполнитель (*Act*) | Выбирает и запускает инструменты. | Tool Router, Sandbox executor, Terminal wrapper |
| Обсервер (*Observation*) | Сохраняет результаты вызова, анализирует отклик/ошибку. | Diff-analyzer, Unit-tester, Lint-parser |
| Память/Контекст | Долгосрочный и оперативный контекст для следующих шагов. | RAG-хранилище кода, векторная БД, кратковременный стек мыслей |
| Менеджер цикла | Решает «завершить или продолжать», ограничивает итерации. | Stopping criteria, Cost-guard, Timeouts |

<div style="text-align: center;">
  <img src="docs/multi_agents.png" alt="alt text" style="width:400px;">
</div>

## Разновидности инструментов

| Категория | Примеры вызовов | Типы задач |
|-----------|-----------------|------------|
| **Файловые** | `readFile`, `writeFile`, `appendFile` | Рефакторинг, вставка кода. |
| **Семантический поиск** | `findSymbol`, `searchVector` | Быстрый RAG по кодовой базе. |
| **Сборка / тесты** | `runTerminalCommand`, `npm test` | Верификация изменений. |
| **Веб-APIs** | `httpRequest`, `getSwaggerSchema` | Интеграции и док-ген. |
| **Кодогенерация** | `generateUnitTest`, `generateMigration` | Шаблоны, вспомогательный код. |
| **Виртуальная песочница** | `pythonBox.exec`, `nodeSandbox.exec` | Безопасное выполнение фрагментов|

### Вызовы и будущее AI-ассистентов

Несмотря на впечатляющий прогресс, у текущих систем есть ограничения:
- **"Галлюцинации"**: Модели могут генерировать несуществующий синтаксис или выдумывать несуществующие функции API.
- **Безопасность**: Агент, имеющий доступ к терминалу и файловой системе, представляет потенциальный риск. Необходимо тщательно контролировать его полномочия.
- **Ограниченный контекст**: Даже с RAG, модели все еще имеют ограниченное "окно внимания" и могут упускать общую картину в очень больших проектах.
- **Высокая стоимость**: Запросы к самым мощным моделям (таким как GPT-4) могут быть дорогостоящими, особенно в агентном режиме, где выполняются десятки итераций.

**Что нас ждет впереди?**
- **Мультимодальность**: Ассистенты, способные понимать диаграммы, скриншоты и даже дизайн-макеты.
- **Самовосстанавливающийся код**: Агенты, которые не только пишут код, но и автоматически исправляют баги на основе отчетов об ошибках.
- **Глубокая персонализация**: Ассистенты, дообученные на вашем личном стиле кодирования и предпочтениях.


## Заключение 
(переходим к следующей части семинара)

Мы рассмотрели, как AI-ассистенты прошли путь от простого автодополнения до сложных систем, способных понимать кодовую базу и автономно выполнять задачи. Мы заглянули под капот ключевых технологий: FIM, AST-анализа, RAG и агентного подхода ReAct.

Теперь, когда у нас есть теоретическая база, на следующей части семинара мы перейдем к самому интересному — **практическому созданию собственного coding agent фреймворка (или же: мультиагентной системы кодогенерации)** с использованием фреймворка AutoGen. Мы научим его планировать, использовать инструменты и решать реальные задачи по разработке автономно.

In [226]:
print('well done')

well done
